In [49]:
import torch
import torch.nn as nn

import torch.nn.functional as F

In [308]:
block_size = 8
nembed = 64
nembed2 = 64

batch_size = 64
device = 'cuda'

In [309]:
text = open("input.txt", 'r').read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
encode = lambda x: [stoi[s] for s in x]
decode = lambda x: "".join([itos[i] for i in x])

data = torch.tensor(encode(text), dtype = torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size - 1, (batch_size, ))
    x = torch.stack([data[i: i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [481]:
class mLSTMCell(nn.Module):
    def __init__(self):
        super().__init__()
        self.qkv= nn.Linear(nembed, nembed2 * 3)
        self.fi = nn.Linear(nembed, 1 * 2)
        self.o = nn.Linear(nembed, nembed2)
    
    def rnn_forward(self, xt, nprev, cprev):
        q, k, v, = self.qkv(xt).split(nembed2, dim = -1) # b, t, c
        i, f = self.fi(xt).split(1, dim = -1)
        o = self.o(xt)
        k = k * (nembed2 ** -0.5)
        
        f = torch.sigmoid(f)
        i = torch.exp(i)
        o = torch.sigmoid(o)
        #cprev = b, c, c
        #nprev = b, t, c

        ct = f[..., None] * cprev + i[..., None] * (v[..., None] * k[..., None, :])
        nt = f * nprev + i * k
        ht_bar = ct @ q[..., None] / torch.abs(nt[..., None, :] @ q[..., None]).clamp(min = 1)
        ht = o * ht_bar[..., -1]
        return ht, nt, ct

    def parallel_forward(self, xt):
        b, t, c = xt.size()
        q, k, v, = self.qkv(xt).split(nembed2, dim = -1) # b, t, c
        i, f= self.fi(xt).split(1, dim = -1)
        f = f.squeeze(-1)
        i = i.squeeze(-1)
        o = self.o(xt)
        k = k * (nembed2 ** -0.5)

        F = torch.zeros(b, t, t).to(q.device)
        for l in range(t):
            for j in range(t):
                if j == l:
                    F[:, l, j] = torch.ones_like(F[:, l, j])
                if l > j:
                    F[:, l, j] = f[:, j + 1: l + 1].sigmoid().prod(1)
        
        I = torch.zeros(b, t, t).to(q.device)
        for l in range(t):
            for j in range(t):
                if l >= j:
                    I[:, l, j] = i[:, j]
        # print(f,'\n',F, '\n',I)
        Cbar = (q @ k.transpose(-2 , -1)) * F * I.exp()
        C = Cbar / torch.abs(Cbar.sum(2, keepdim = True)).clamp(min = 1)
        hbar = C @ v
        return hbar * o.sigmoid()


In [490]:
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.start = nn.Parameter(torch.zeros(1, nembed2))
        self.cstart = nn.Parameter(torch.zeros(1, nembed2, nembed2))
        self.emb = nn.Embedding(vocab_size, nembed)
        self.cell = mLSTMCell()
        self.proj = nn.Linear(nembed2, vocab_size)
    
    def forward(self, idx, targets = None, tp = 0):
        B, T = idx.size()
        emb = self.emb(idx)
        #sequentially iterate over inputs and update torche rnn state each tick
        if tp:
            nprev = self.start.expand((B, -1, -1))
            cprev = self.cstart.expand((B, -1, -1, -1))
            hiddens = []
            for i in range(T):
                xt = emb[:, i: i + 1, :]
                # xt = nn.LayerNorm(nembed).cuda()(xt)
                ht, nt, ct = self.cell.rnn_forward(xt, nprev, cprev)
                nprev, cprev = nt, ct
                hiddens.append(ht)
            hiddens = torch.cat(hiddens, 1)
        else:
            hiddens = self.cell.parallel_forward(emb)
        logits = self.proj(hiddens)

        loss = None
        if targets is not None:
            loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

In [491]:
model = RNN().to(device)
print("Model Parameters: ", sum(p.numel() for p in model.parameters()))

Model Parameters:  29315


In [ ]:
max_iters = 3000
eval_iters = 50
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

from tqdm import tqdm
pbar = tqdm(range(max_iters))

for j in pbar:
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb, tp = 0)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none = True)
    pbar.set_description(f"Loss {loss.item():.4f}")
    # if j % 50== 0:
    #     losses = estimate_loss()
    #     print(f"\nStep {j}: train_loss {losses['train']:.4f}  val_loss {losses['val']:.4f}")

In [520]:
model(xb)[0][1][5], model(xb, tp = 1)[0][1][5]

(tensor([ 0.3245,  3.5249,  0.0480, -3.0780, -3.5234,  0.5846,  1.5399, -1.2726,
          0.8633, -3.1045,  0.3755,  0.1603, -1.2028, -3.2946, -4.1614, -3.6202,
         -4.0068, -2.8602, -3.7481, -3.5890, -4.8860, -2.9233, -4.5777, -4.2971,
         -3.7921, -4.0429, -3.4573, -3.0167, -4.5481, -3.8404, -3.1294, -3.6817,
         -3.1601, -2.5103, -3.7956, -3.5982, -3.8480, -3.4387, -4.1663,  5.0372,
         -2.8464, -0.7980,  0.1525,  5.7629, -2.5494, -1.6712,  1.0083,  4.5786,
         -3.6007, -0.4976, -0.5019, -0.3955, -1.9380,  4.3374, -1.6218, -2.7170,
          0.6193,  0.7554,  0.4953,  1.7111, -2.0904, -2.0211, -2.9157,  2.0636,
         -3.3485], device='cuda:0', grad_fn=<SelectBackward0>),
 tensor([ 0.3245,  3.5249,  0.0480, -3.0780, -3.5234,  0.5846,  1.5399, -1.2726,
          0.8633, -3.1045,  0.3755,  0.1603, -1.2028, -3.2946, -4.1614, -3.6202,
         -4.0068, -2.8602, -3.7481, -3.5890, -4.8860, -2.9233, -4.5777, -4.2971,
         -3.7921, -4.0429, -3.4573, -3.0167, 

In [368]:
@torch.no_grad()
def generate(model, idx, max_new_tokens):
    for _ in range(max_new_tokens):
        idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]
        logits, _ = model(idx_cond, tp = 1)
        logits = logits[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx


In [369]:
print(decode((generate(model, torch.tensor([[0]], dtype = torch.long, device= 'cuda'), 100))[0].tolist()))


Any asshreand hse uouln,'s my wis wertYEa wever, s asearttrust. teas, ver thato?
To eswoungtot; fo o
